# Projet python pour la data science : détection de la langue d'un texte grâce aux chaînes de Markov

Lors de ce projet python, nous avons décidé de prédire la langue d'un texte grâce aux chaînes de markov sur la base du chapitre 22 du livre "Intelligence artificielle" de Stuart Russel et Peter Norvig.

Pour récupérer des corpus de textes dans différentes langues, nous avons décidé de scrapper des articles Wikipédia grâce à son API (https://fr.wikipedia.org/w/api.php). Nous avons utilisé 10 langues différentes pour essayer notre modèle :
- Français
- Esapagnol
- Anglais
- Allemand
- African
- Italien
- Néerlandais
- Polonais
- Catalan
- Suédois

## Première étape : La récupération des données

Dans un premier temps, nous utilisons l'API wikipédia pour récupérer les titres des articles dans chaque langue. Pour cela nous utilisons la bibliothèque request. L'extrait de code suivant permet de récupérer 3 titre d'articles wikipédia en français commençant par la lettre "b".

In [2]:
import requests

S = requests.Session()

PARAMS = {
            "action": "opensearch",
            "namespace": "0",
            "search": "b",
            "limit": "3",
            "format": "json"
        }

URL = "https://fr.wikipedia.org/w/api.php"

response = S.get(url=URL, params=PARAMS)
print(response.json()[1])

['B', 'Belgique', 'Brésil']


Une fois le nom de la page récupérée, on va récupérer le texte de la page qui porte ce titre :

In [3]:
params = {
                "action": "query",
                "format": "json",
                "titles": "Belgique",
                "prop": "extracts",
                "explaintext": True}
response = requests.get(url=URL, params=params)

data = response.json()
print(list(data["query"]["pages"].values())[0]["extract"])

La Belgique (/bɛlʒik/  ; en néerlandais : België /ˈbɛlɣiǝ/  ; en allemand : Belgien /ˈbɛlgiən/ ), en forme longue le royaume de Belgique, est un pays d’Europe de l'Ouest, bordé par la France, les Pays-Bas, l’Allemagne, le Luxembourg et la mer du Nord. Politiquement, il s'agit d’une monarchie constitutionnelle fédérale à régime parlementaire. Elle est l’un des six pays fondateurs de l’Union européenne et accueille, dans sa capitale Bruxelles, le Conseil de l'Union européenne et le Conseil européen, la Commission européenne, les Commissions parlementaires et six sessions plénières additionnelles du Parlement européen, ainsi que d’autres organisations internationales comme l’OTAN. Le pays accueille également, à Mons, le Grand Quartier général des puissances alliées en Europe (SHAPE). La Belgique couvre une superficie de 30 688 km2 avec une population de 11 507 163 habitants au 1er janvier 2021, soit une densité de 373,97 habitants/km2.
Située à mi-chemin entre l’Europe germanique et l’Eur

Une fois le contenu d'un suffisement grand nombre de pages récupérés, nous pouvons commmencer à créer des chaines de Markov différentes pour chaque langue. Pour entraîner notre modèle, nous avons récupéré 390 articles par langues. Les articles ne sont pas nécessairement les même s'une langue à l'autre car nous avons demandé 15 article commençant par chaque lettre de l'alphabet dans chaque langue, hors les articles n'arrivent pas nécessairement dans le même ordre alphabétique dans chaque langue. Voici le nombre de caractères que nous avons dans nos corpus d'entraînement :

- pl 13.541.458 
- fr 23.716.203 
- es 22.073.000 
- it 17.310.326 
- de 25.182.231 
- ca 14.008.554 
- sv 9.597.557 
- af 10.104.721 
- nl 12.651.444 


## Les modèles Unigrammes Bigrammes et Trigrammes :

Nous allons détailler dans cette partie le principe des modèles dit n-grammes. Pour celà, on considère un alphabet donnée constitué de $k$ caractères. Dans notre cas, nous avons considéré les caractère du format d'encodage ASCII, et avont gardé uniquement les lettres de l'aphalbets minuscules,les lettres minuscules avec accents, les espaces et nous avons remplacés tous les autres caractères par un seul et même caractère pour gagner en simplicité et réduire la taille de nos modèles (soit un total de 60 carcatères).

### Le modèle unigrammes

Dans le modèle unigramme, on regarde juste la fréquence d'apparition de chaque lettre. On obtient donc un vecteur de probabilité de présence de chaque caractère dont les composantes somment à 1. Pour prédire la langue d'un texte, on observe donc la fréquence d'appartion de chaque lettre et on obtient un nouveau vecteur. On mesure la distance euclidienne entre ce vecteur et les vecteurs de fréquence que nous avons observé sur nos corpus d'entraînement. On considère que la langue du texte est la langue du vecteur avec lequel la distance est la plus petite.

Cette méthode est très imparfaite car elle ne prend pas en compte les enchaînements de lettre (par exemple l'enchainement "th" est bien plus fréquent en anglais qu'en français). Voici les résultats que nous avons obtenus pour le modèle unigramme :





In [ ]:
#mettre ici des scoring pour le modèle unigrammes

### Le modèle Bi-grammes

Le modèle Bi-grammes utilise les chaînes de markov afin de gagner en précision par rapport au précédent modèle. On considère dans ce modèle que la probabilité d'apparition d'un carcatère dépend de la lettre d'avant. Notre chaîne de markov sera donc une chaîne à $k$ états. Les probabilités de transitions d'un état à un autre seront différentes pour chaque langue. 

#### En théorie

Pour un texte donné $T = c_0c_1....c_n$ constitué de $n$ carctères, on va calculer la probabilité que le texte soit généré par une langue sachant qu'il est dans cette langue. On va donc calculer $\mathcal{P}(c_0....c_n | \text{langue}) = \mathcal{P}(c_0|\text{langue}) \prod_{i=1}^{n} \mathcal{P}(c_i | c_{i-1}, \text{langue})$. On considère que la langue du texte est la langue qui nous permet d'obtenir la probabilité la plus élevée dans la formule précédente.

#### En pratique 

Dans la pratique, les probabilités deviennent très vites très petites et nous atteignons le zero machine bien trop rapidement. Pour estimer la probabilité d'occurence d'un texte, on applique donc cette formule sur chaque mots (suite de caractères entre deux espaces) du texte, on somme toutes ces probabilités et on divise par le nombre de mot. On obtient ainsi la probabilité moyenne de présence de chaques mots dans le texte.

#### Les résultats

Voici les résultats que nous avons obtenus avec notre modèle bigramme :

In [ ]:
# mettre ici des benchmarks pour le modèle bigrammes

## Le modèle Trigramme

Le principe du modèle trigramme est le même que pour le modèle bigramme, cependant on utilise cette fois les deux caractères précédents pour prédire le caractère suivant. On se retrouve donc avec une chaîne de markov avec $k^2$ états.

## Statistiques descriptive

Nous avons décidé d'utiliser la data visualisation afin de facilement observer les différences entre les langues. Par exemple dans le modèle unigramme, nous pouvons voir que les fréquences ne sont pas les même selon les langues 
<span style="color: #26B260"> rajouter des images pour comparer </span>

De même pour le modèle bigramme où l'on peut voir les probabilités de trasition d'une lettre à une autre :

![Texte alternatif](/home/onyxia/work/projet_python_ds/visualization/matrix_visualization/Matrix_viz_fr_zoomed.png "Matrice de transition du modèle de markov associé au français")

![Texte alternatif](/home/onyxia/work/projet_python_ds/visualization/matrix_visualization/Matrix_viz_en_zoomed.png "Matrice de transition du modèle de markov associé à l'anglais")
